<a href="https://colab.research.google.com/github/mohamedalaaaz/testpytroch/blob/main/physics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the neural network model
class PhysicsNet(nn.Module):
    def __init__(self):
        super(PhysicsNet, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(1, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 1)
        )

    def forward(self, t):
        return self.net(t)

# Define the physics-informed loss function
def physics_loss(model, t, k=1.0):
    t.requires_grad = True
    y_pred = model(t)

    # Compute dy/dt using autograd
    dy_dt = torch.autograd.grad(y_pred, t, grad_outputs=torch.ones_like(y_pred),
                                retain_graph=True, create_graph=True)[0]

    # The differential equation: dy/dt = -k * y
    residual = dy_dt + k * y_pred
    return torch.mean(residual**2)

# Training loop
def train(model, epochs=2000, lr=0.001):
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        t = torch.linspace(0, 5, 100).view(-1, 1)
        loss = physics_loss(model, t)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if epoch % 200 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item():.6f}")

    return model

# Instantiate and train the model
model = PhysicsNet()
trained_model = train(model)


Epoch 0, Loss: 0.210172
Epoch 200, Loss: 0.000010
Epoch 400, Loss: 0.000006
Epoch 600, Loss: 0.000003
Epoch 800, Loss: 0.000001
Epoch 1000, Loss: 0.000000
Epoch 1200, Loss: 0.000000
Epoch 1400, Loss: 0.000000
Epoch 1600, Loss: 0.000000
Epoch 1800, Loss: 0.000000
